In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt 
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import seaborn as sns 
from scipy import stats
from scipy.stats import pearsonr, ttest_rel

import multiprocessing as mp
import pickle 
import warnings 
warnings.filterwarnings('ignore')

from imports import*
from utils import *
from logistic_regression import *
from rnn import *

from hybrid_sim import *
from hybrid_fit import *
from hybrid_predict import *

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)



In [ ]:
# load data
df_tst = pd.read_csv('../data/TST.csv')
df_iq = pd.read_csv('../data/wasi.csv')
ids = df_iq['SubjectID'].values

def update_data_frame(df): 
    # this funcation add to the data frame 3 columns:
    # 1 - prev_reward : if the last trail was rewarded
    # 2 - transation_prev : if the last transation was rare or common
    # 3 - stay probs 
    df['prev_reward'] = df['reward'].shift(1,fill_value=0)
    df['transition_prev'] = df['transition_type'].shift(1,fill_value=0)
    df['stay'] = (df['action_stage_1'].shift(1)==df['action_stage_1']).astype(int)
    
def sigmoid(x):
    return(1.0 / (1.0 + np.exp(-x)))

def inverse_sigmoid(y):
    return(np.log(y/(1-y)))

all_dfs = []

for i in ids:
    if i == 25510:
        continue
    df = df_tst[df_tst['subjectID'] == i]
    df.reset_index(inplace=True)
    all_dfs.append(df)

new_all_dfs = []

for i in all_dfs:
    if i.shape[0] == 322:
        new_all_dfs.append(i)
        
block_0 = []

for i in new_all_dfs:
    df = i[i['measurement'] == 'baseline']
    df.reset_index(inplace=True)
    block_0.append(df)

block_1 = []

for i in new_all_dfs:
    df = i[i['measurement'] == 'followup']
    df.reset_index(inplace=True)
    block_1.append(df)
    
ids = []
for i in block_0:
    ids.append(i['subjectID'].unique()[0])

IQs = []
for i in ids:
    IQs.append(df_iq[df_iq['SubjectID'] == i].IQ.values[0])
    
for df_i in block_0:
    df_i.rename(columns=
                    {
                    'choice1':'action_stage_1',
                    'choice2':'action_stage_2',
                    '2nd_stage_state':'state_of_stage_2',
                    'transition':'transition_type',
                    'reward':'reward'},inplace=True
                    )
    
    df_i.drop(columns={
        'level_0', 'index','measurement','key1', 'key2','rt1', 'rt2',
       'iti','1st_stage_stim_left', '1st_stage_stim_right',
       '2nd_stage_stim_left', '2nd_stage_stim_right',
         'p1','p2', 'p3', 'p4'},inplace=True)
    
    df_i['action_stage_1']-=1
    df_i['action_stage_2']%=2
    df_i['state_of_stage_2']-=1
    update_data_frame(df_i)
    df_i.dropna(inplace=True)
    
for df_i in block_1:
    df_i.rename(columns=
                    {
                    'choice1':'action_stage_1',
                    'choice2':'action_stage_2',
                    '2nd_stage_state':'state_of_stage_2',
                    'transition':'transition_type',
                    'reward':'reward'},inplace=True
                    )
    
    df_i.drop(columns={
        'level_0', 'index','measurement','key1', 'key2','rt1', 'rt2',
       'iti','1st_stage_stim_left', '1st_stage_stim_right',
       '2nd_stage_stim_left', '2nd_stage_stim_right',
         'p1','p2', 'p3', 'p4'},inplace=True)
    
    df_i['action_stage_1']-=1
    df_i['action_stage_2']%=2
    df_i['state_of_stage_2']-=1
    update_data_frame(df_i)
    df_i.dropna(inplace=True)
    
sns.displot(np.array(IQs))
print('mean',np.array(IQs).mean())
print('std',np.array(IQs).std())

In [ ]:
def train_model_pre(net, train_loader, val_loader , epochs, lr):
    
    min_loss_v = 1000
        
    train_loss, train_ll = np.zeros(epochs), np.zeros(epochs)
    val_loss ,val_ll = np.zeros(epochs), np.zeros(epochs)

    # move net to GPU
    net.to(device)
    # Use Adam optimizer
    optimizer = optim.Adam(net.parameters(), lr=lr) 
    criterion = nn.BCELoss()
    
    # Loop over epochs
    for i in tqdm(range(epochs)):
        
        # Loop over training batches
        running_loss_tr = []
        for j,(X_train,y_train) in enumerate(train_loader):
            
            # move to GPU
            X_train , y_train = X_train.to(device), y_train.to(device)
            # reshape to 1 X batch_size X input_size
            X_train = X_train.reshape(1,X_train.shape[0], INPUT_SIZE)
            # zero the gradient buffers
            optimizer.zero_grad() 
            out, hn = net(X_train)
            # Reshape to (SeqLen x Batch, OutputSize)
            out = out.view(-1, OUTPUT_SIZE)
            loss = criterion(out, y_train)
            loss.backward()
            optimizer.step() # Does the update
            running_loss_tr.append(loss.item())
            
        train_loss[i], train_ll[i] = eval_net(net, train_loader)
        val_loss[i], val_ll[i] = eval_net(net, val_loader)
        
        if val_loss[i] <= min_loss_v:
            checkpoint = {'epoch':i+1,'model_state':net.state_dict(),
                          'optim_state':optimizer.state_dict(),'loss':val_loss[i]}
            torch.save(checkpoint,f'rnn_finetune_5.pth')
            min_loss_v = val_loss[i]
        
        print('train loss: ', train_loss[i])
        print('val loss: ',val_loss[i])
        net.train()

    return net, train_loss , train_ll , val_loss, val_ll

In [ ]:
df_train = pd.concat(block_1).reset_index()
df_val = pd.concat(block_0).reset_index()

INPUT_SIZE = 5
OUTPUT_SIZE = 2
LERANING_RATE = 0.001

hidden_size = 5
num_layers = 1
epochs = 500


train_data = behavior_dataset(df_train)
val_data = behavior_dataset(df_val)

train_loader = DataLoader(train_data,shuffle=False,batch_size=1000)
val_loader = DataLoader(val_data,shuffle=False,batch_size=1000)

rnn = GRU_NN(INPUT_SIZE, hidden_size, num_layers, OUTPUT_SIZE)
rnn, train_loss, train_ll, val_loss, val_ll, = train_model_pre(rnn,
                                                            train_loader,
                                                            val_loader,
                                                            epochs=epochs,
                                                            lr=LERANING_RATE) 




plt.plot(train_loss)
plt.plot(val_loss)

In [ ]:
# load data
df_tst = pd.read_csv('../data/TST.csv')
df_iq = pd.read_csv('../data/wasi.csv')
ids = df_iq['SubjectID'].values

def update_data_frame(df): 
    # this funcation add to the data frame 3 columns:
    # 1 - prev_reward : if the last trail was rewarded
    # 2 - transation_prev : if the last transation was rare or common
    # 3 - stay probs 
    df['prev_reward'] = df['reward'].shift(1,fill_value=0)
    df['transition_prev'] = df['transition_type'].shift(1,fill_value=0)
    df['stay'] = df['action_stage_1'].shift(1)==df['action_stage_1']
    
def sigmoid(x):
    return(1.0 / (1.0 + np.exp(-x)))

def inverse_sigmoid(y):
    return(np.log(y/(1-y)))

all_dfs = []

for i in ids:
    if i == 25510:
        continue
    df = df_tst[df_tst['subjectID'] == i]
    df.reset_index(inplace=True)
    all_dfs.append(df)

new_all_dfs = []

for i in all_dfs:
    if i.shape[0] == 443: # 121 322 443
        new_all_dfs.append(i)
        
block_00 = []

for i in new_all_dfs:
    df = i[i['measurement'] == 'baseline']
    df.reset_index(inplace=True)
    block_00.append(df)

block_11 = []

for i in new_all_dfs:
    df = i[i['measurement'] == 'followup']
    df.reset_index(inplace=True)
    block_11.append(df)

block_22 = []

for i in new_all_dfs:
    df = i[i['measurement'] == 'six_month']
    df.reset_index(inplace=True)
    block_22.append(df)
    
ids = []
for i in block_00:
    ids.append(i['subjectID'].unique()[0])

IQs_2 = []
for i in ids:
    IQs_2.append(df_iq[df_iq['SubjectID'] == i].IQ.values[0])
    
for df_i in block_00:
    df_i.rename(columns=
                    {
                    'choice1':'action_stage_1',
                    'choice2':'action_stage_2',
                    '2nd_stage_state':'state_of_stage_2',
                    'transition':'transition_type',
                    'reward':'reward'},inplace=True
                    )
    
    df_i.drop(columns={
        'level_0', 'index','measurement','key1', 'key2','rt1', 'rt2',
       'iti','1st_stage_stim_left', '1st_stage_stim_right',
       '2nd_stage_stim_left', '2nd_stage_stim_right',
         'p1','p2', 'p3', 'p4'},inplace=True)
    
    df_i['action_stage_1']-=1
    df_i['action_stage_2']%=2
    df_i['state_of_stage_2']-=1
    update_data_frame(df_i)
    df_i.dropna(inplace=True)
    
for df_i in block_11:
    df_i.rename(columns=
                    {
                    'choice1':'action_stage_1',
                    'choice2':'action_stage_2',
                    '2nd_stage_state':'state_of_stage_2',
                    'transition':'transition_type',
                    'reward':'reward'},inplace=True
                    )
    
    df_i.drop(columns={
        'level_0', 'index','measurement','key1', 'key2','rt1', 'rt2',
       'iti','1st_stage_stim_left', '1st_stage_stim_right',
       '2nd_stage_stim_left', '2nd_stage_stim_right',
         'p1','p2', 'p3', 'p4'},inplace=True)
    
    df_i['action_stage_1']-=1
    df_i['action_stage_2']%=2
    df_i['state_of_stage_2']-=1
    update_data_frame(df_i)
    df_i.dropna(inplace=True)
    

for df_i in block_22:
    df_i.rename(columns=
                    {
                    'choice1':'action_stage_1',
                    'choice2':'action_stage_2',
                    '2nd_stage_state':'state_of_stage_2',
                    'transition':'transition_type',
                    'reward':'reward'},inplace=True
                    )
    
    df_i.drop(columns={
        'level_0', 'index','measurement','key1', 'key2','rt1', 'rt2',
       'iti','1st_stage_stim_left', '1st_stage_stim_right',
       '2nd_stage_stim_left', '2nd_stage_stim_right',
         'p1','p2', 'p3', 'p4'},inplace=True)
    
    df_i['action_stage_1']-=1
    df_i['action_stage_2']%=2
    df_i['state_of_stage_2']-=1
    update_data_frame(df_i)
    df_i.dropna(inplace=True)
    
all_blocks = [block_00,block_11,block_22] 

sns.displot(np.array(IQs_2))
print('mean',np.array(IQs_2).mean())
print('std',np.array(IQs_2).std())

In [ ]:
# num of agent
N = len(all_blocks[0])

# num of block
num_of_block = 3

# for cross valdation 
array = np.arange(num_of_block)
cv = [np.roll(array,i) for i in range(num_of_block)]
cv = np.array(cv)


INPUT_SIZE = 5
OUTPUT_SIZE = 2
LERANING_RATE = 0.001

hidden_size = 5
num_layers = 1
epochs = 100

loss_train, loss_val, loss_test  = [], [], []
ll_train, ll_val, ll_test = [], [], []

for n in tqdm(range(N)):
    for train, val, test in cv:

        train_data = behavior_dataset(all_blocks[train][n])
        val_data = behavior_dataset(all_blocks[val][n])
        test_data = behavior_dataset(all_blocks[test][n])

        train_loader = DataLoader(train_data,shuffle=False,batch_size=len(train_data))
        val_loader = DataLoader(val_data,shuffle=False,batch_size=len(val_data))
        test_loader = DataLoader(test_data,shuffle=False,batch_size=len(test_data))
        
        load = torch.load(f'rnn_finetune_5.pth')
        model = GRU_NN(INPUT_SIZE, hidden_size, num_layers, OUTPUT_SIZE)
        model.load_state_dict(load['model_state'])
        
        _, train_loss, train_ll, val_loss, val_ll, test_loss, test_ll = train_model(model,
                                                                                train_loader,
                                                                                val_loader,
                                                                                test_loader,
                                                                                epochs=epochs,
                                                                                lr=LERANING_RATE) 
                                                                                                                                       
        loss_train.append(train_loss)
        loss_val.append(val_loss)
        loss_test.append(test_loss)
        
        ll_train.append(train_ll)
        ll_val.append(val_ll)
        ll_test.append(test_ll)
        
    print('Done agent',n)
    
    
with open('../results_finetune/loss_train.pickle', 'wb') as handle:
    pickle.dump(loss_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results_finetune/loss_val.pickle', 'wb') as handle:
    pickle.dump(loss_val, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results_finetune/loss_test.pickle', 'wb') as handle:
    pickle.dump(loss_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results_finetune/ll_train.pickle', 'wb') as handle:
    pickle.dump(ll_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../results_finetune/ll_val.pickle', 'wb') as handle:
    pickle.dump(ll_val, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('../results_finetune/ll_test.pickle', 'wb') as handle:
    pickle.dump(ll_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
        